# Lab - 8 : Attention Mechanism using LSTM
---
- Namansh Singh Maurya
- 22MIA1034



Workflow to this Experiment

1.  Load the dataset
2.  Padding Sequences
3.  Define the i/p layer
4.  Embedding
5.  LSTM
6.  Attention Layer (Attention Mechanism)
7.  Fully Connected Layer (FFN)
8.  Compile
9.  Training
10. Testing


#### Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation, Layer
import numpy as np

#### Loading the imdb dataset


In [ ]:
vocab_size = 1000
max_len = 200
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words = vocab_size)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


#### Padding to make all the sentences to same length

In [ ]:
x_train = pad_sequences(x_train,maxlen=max_len,padding='post')
x_test = pad_sequences(x_test,maxlen=max_len,padding='post')

#### Defining input layer

In [ ]:
input_layer = tf.keras.layers.Input(shape=(max_len,))

#### Defining Embedding Layer

In [ ]:
embedding = Embedding(input_dim=vocab_size,output_dim=128,input_length=max_len)(input_layer)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


#### Defining LSTM Layer

In [ ]:
lstm_output = LSTM(64,return_sequences=True)(embedding)

#### Attention Mechanism

In [ ]:
class AttentionLayer(Layer):
  def __init__(self, **kwgrs):
    super(AttentionLayer, self).__init__(**kwgrs)
    self.w = tf.keras.layers.Dense(32, activation='tanh')
    self.v = tf.keras.layers.Dense(1, activation=None)

  def call(self,lstm_output):
    w = self.w(lstm_output)
    v = self.v(w)
    attention_scores = tf.nn.softmax(v,axis=1)
    context_vector = tf.reduce_sum(attention_scores*lstm_output, axis=1)
    return context_vector

In [ ]:
context_vector = AttentionLayer()(lstm_output)

In [ ]:
output = Dense(1, activation='sigmoid')(context_vector)

In [ ]:
model = tf.keras.Model(inputs = input_layer, outputs = output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Training the Model

In [ ]:
model.fit(x_train,y_train,batch_size=64,epochs=3,validation_data=(x_test,y_test))

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.6796 - loss: 0.5551 - val_accuracy: 0.8573 - val_loss: 0.3345
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.8567 - loss: 0.3320 - val_accuracy: 0.8648 - val_loss: 0.3144
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.8805 - loss: 0.2919 - val_accuracy: 0.8709 - val_loss: 0.3010


#### Testing the model

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8694 - loss: 0.3051
Test accuracy: 0.8708800077438354
